# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_env_model import * #PPO_Agent_v2 PPO_Agent_with_Guided_AC
from ENV_DETAILS import *
from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

2024-05-08 08:59:07.997293: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Selected port: 58143


2024-05-08 08:59:10.450353: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:58143/ (Press CTRL+C to quit)


In [2]:
ENV = "Pendulum-v1"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]

EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env

<TimeLimit<OrderEnforcing<PendulumEnv<Pendulum-v1>>>>

In [4]:
observation = env.reset()

# Initialize an empty list to store rewards
rewards = []

# Run episodes and collect rewards
for episode in range(100000):
    done = False
    while not done:
        observation, reward, done, info = env.step(env.action_space.sample())
        rewards.append(reward)

# Calculate normalization factor (e.g., mean or standard deviation)
mean_reward = np.mean(rewards)
std_dev_reward = np.std(rewards)
mean_reward, std_dev_reward

(-5.630153435842235, 3.9406927377157532)

In [5]:
s = env.observation_space.sample()
s

array([ 0.7181412, -0.7727178,  4.244884 ], dtype=float32)

In [6]:
env.observation_space, env.observation_space.shape[0]

(Box([-1. -1. -8.], [1. 1. 8.], (3,), float32), 3)

In [7]:
s = env.reset()
s[0]

-0.70830816

In [8]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

### Training/Hyperparam run

In [9]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "logs_hyper/fit_PPO_env_model/"

In [10]:
if HYPERPARAM_TUNING: 
    
    # Trial id :08 | Score :-71.19572300865248 --> {'lr_actor_critic': 0.0001443924191964091, 'dense_units_act_crit_0': 138, 'dense_units_act_crit_1': 210, 'kl_divergence_target': 0.05}
    # Trial id :02 | Score :-72.41986896912546 --> {'lr_actor_critic': 0.000829844022383942, 'dense_units_act_crit_0': 234, 'dense_units_act_crit_1': 215, 'kl_divergence_target': 0.1}
    # Trial id :03 | Score :-74.04910396664376 --> {'lr_actor_critic': 0.000987130392464792, 'dense_units_act_crit_0': 255, 'dense_units_act_crit_1': 188, 'kl_divergence_target': 0.1}

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo_env_model"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo_env_model/", evaluation_epoch = env._max_episode_steps, training_steps = 1000000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.9, discount_max = 0.99, 
                #discount= 0.90,#WAS THIS ONE
                #gae_factor = 0.95, #WAS THIS ONE
                gae_min = 0.85, gae_max = 0.96, 
                policy_clip =0.2,
                lr_actor_crit_min = 0.00001, lr_actor_crit_max = 0.001,
                #entropy_factor = 0.05, 
                entropy_factor_min = 0.001, entropy_factor_max =  0.5,
                lr_model_min = 0.00001, lr_model_max =  0.001, kl_divergence_target = None,#0.1,#
                #dense_layers = [128,128],
                dense_min = 32, dense_max = 150,
                environment_name=ENV, num_layers_act = 2, #max_num_layers_act = 2
                num_layers_model = 1, #WAS THIS ONE
                training_epoch = 1,
                memory_size = env._max_episode_steps, 
                normalize_reward=False, normalize_advantage= False,
                scaling_factor_reward = 0.1,
                reward_norm_factor = 16,
                #memory_size_max= env._max_episode_steps
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 40,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name,
            #seed=0
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")

        dense_units_act_crit = [128,64]
        num_layers_actor_critic = len(dense_units_act_crit)
        num_layer_m = None
        dense_units_model = []

        
        model = run_training(training_steps = 500000,
                            discount = 0.99, 
                            dense_units_act_crit = dense_units_act_crit,  
                            dense_units_model = dense_units_model,  
                            num_layer_a_c = num_layers_actor_critic,  
                            num_layer_m = num_layer_m, 
                            writer = writer, 
                            environment_name = ENV, 
                            return_agent = True, 
                            lr_actor_critic= 0.0001,  
                            lr_model = None,
                            sucess_criteria_epochs=SUCESS_CRITERIA_EPOCH, 
                            sucess_criteria_value = SUCESS_CRITERIA_VALUE, 
                            gae_lambda = 0.95, 
                            entropy_coeff = 0.05, 
                            policy_clip = 0.2, training_epoch = 20, 
                            scaling_factor_reward = 0.1, 
                            kl_divergence_target = 0.01,
                            memory_size =  env._max_episode_steps,
                            normalize_reward=False, normalize_advantage= False, use_mlflow = False)


Trial 11 Complete [01h 23m 50s]
total_train_reward: -1224.7845143625848

Best total_train_reward So Far: -582.1180362811813
Total elapsed time: 1d 05h 32m 00s

Search: Running Trial #12

Value             |Best Value So Far |Hyperparameter
0.93              |0.91              |discount
0.9               |0.95              |gae_lambda
0.00052037        |0.00043567        |lr_actor_critic
0.00024119        |0.00022614        |lr_model
0.17937           |0.019308          |entropy_coeff
92                |98                |dense_units_act_crit_0
134               |118               |dense_units_act_crit_1
111               |63                |n_dense_layers_model0
0.11923           |0.08225           |kl_divergence_target

Trial number :  12
Epoch: 2000 : Reward eval/Train: -1434.0959992726632/-1445.5199206754312 
Epoch: 4000 : Reward eval/Train: -1462.5710875569798/-1536.1319632258242 
Epoch: 6000 : Reward eval/Train: -1536.7572123636953/-1539.4081192122342 
Epoch: 8000 : Reward eval/Tr

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]

for trials in tuner.oracle.get_best_trials(num_trials=5):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:
for best_hps in tuner.get_best_hyperparameters(num_trials=2):
    #env_model = tuner.get_best_models()[0]
    print(f"Best Hyperparameters: {best_hps.__dict__}")
    env_model = tuner.hypermodel.build(best_hps)  # Build the model with best hyperparameters
    env_model = rerun_training(training_steps  = 1500000, 
                            model = env_model
                            )

In [ ]:
val_env = gym.make(ENV)#, render_mode = "rgb_array"
# dir = r"Hyperparam_kt_ppo"
# for trials in tuner.oracle.get_best_trials(num_trials=1):
#     print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)    
#     training_steps = 1000000
#     entropy_factor = trials.hyperparameters.values["entropy_coeff"]
#     discount = trials.hyperparameters.values["discount"]
#     gae = trials.hyperparameters.values["gae_lambda"]
#     policy_clip = 0.2#trials.hyperparameters.values["policy_clip"]
#     scaling_factor_reward= 0.1#trials.hyperparameters.values["scaling_factor_reward"]
    
    
#     lr_actor=  trials.hyperparameters.values["lr_actor_critic"]
#     lr_critic=  None#trials.hyperparameters.values["lr_critic"]
#     lr_model=  None#trials.hyperparameters.values["lr_model"]
    
#     try:
#         n_dense_layers_actor = trials.hyperparameters.values["n_dense_layers_actor"]
#     except : 
#         n_dense_layers_actor = 1
        
#     try:
#         n_dense_layers_critic = trials.hyperparameters.values["n_dense_layers_critic"]
#     except:
#         n_dense_layers_critic = 1
        

#     dense_layers_actor = []
#     for i in range(n_dense_layers_actor):
#         dense_layers_actor.append(trials.hyperparameters.values['dense_units_act_crit_'+str(i)])

#     dense_layers_critic = []
#     # for i in range(n_dense_layers_critic):
#     #     dense_layers_critic.append(trials.hyperparameters.values['dense_units_crit_'+str(i)])

    
#     n_dense_layers_model = 1
#     dense_layers_model = []
#     for i in range(n_dense_layers_model):
#         dense_layers_model.append(trials.hyperparameters.values['n_dense_layers_model'+str(i)])

#     model = run_training(
#         training_steps = training_steps,   
#             discount = discount,
#             dense_units_act = dense_layers_actor, 
#             dense_units_crit = dense_layers_critic,
#             dense_units_model = dense_layers_model,
#             num_layer_a = n_dense_layers_actor,
#             num_layer_c = n_dense_layers_critic,
#             num_layer_m = n_dense_layers_model,
#             writer = writer,  
#             save_factor=50000, 
#             sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, 
#             sucess_criteria_value = SUCESS_CRITERIA_VALUE,
#             environment_name=ENV,
#             evaluation_epoch = env._max_episode_steps,
#             return_agent = True,
#             lr_actor= lr_actor, 
#             lr_critic= lr_critic,
#             lr_model= lr_model,
#             gae_lambda=gae,
#             training_epoch= 200,
#             entropy_coeff= entropy_factor,
#             policy_clip = policy_clip,
#             memory_size= env._max_episode_steps,
#             id = int(trials.trial_id),
#             scaling_factor_reward = scaling_factor_reward)
        
#     break


def final_evaluation(eval_model, eval_env, n_tries=1, exploration ="soft", video_name = "./PPO_soft_video.mp4", sucess_criteria_epochs= 100):
    rewards_history = []
    log_dir = "Evaluation_process/A3C_"+str(exploration)+"/" +  datetime.now().strftime("%Y%m%d-%H%M%S")
    tb_summary_writer = tf.summary.create_file_writer(log_dir)

    for k in range(n_tries):

        if k == 0 : video = VideoRecorder(eval_env, path=video_name)

        obs = eval_env.reset()
        total_reward = 0

        log_dir_trial = "Evaluation_process/A3C_trial_"+str(exploration)+"/" + str(k)+ datetime.now().strftime("%Y%m%d-%H%M%S")
        tb_summary_writer_trial = tf.summary.create_file_writer(log_dir_trial)
        
        epoch = 0
        while(True):
    
            if k == 0 : video.capture_frame()
            actions, _, _, _ ,_= eval_model.actor_critic(obs.reshape((1,eval_model.obs_shape[0])))
                     
            obs, reward, done , info = eval_env.step(actions)
            total_reward += reward
            #done = truncated or terminated  #terminated, truncated 

            with tb_summary_writer_trial.as_default():
                tf.summary.scalar('Final_eval_rewards', total_reward, step=int(epoch) )
                tf.summary.scalar('Final_eval_state', obs[0], step=int(epoch) )
                tf.summary.scalar('Final_eval_velocity', obs[1], step=int(epoch) )
                
            if done:
                break

            epoch +=1

        rewards_history.append(total_reward)
        with tb_summary_writer.as_default():
            tf.summary.scalar('Rewards_history', total_reward, step=int(k) )

            if len(rewards_history) >sucess_criteria_epochs:
                tf.summary.scalar('Average_rewards_history', np.mean(rewards_history[-sucess_criteria_epochs:]), step=int(k) )

        eval_env.close()
        if k == 0 : video.close()

    return np.mean(rewards_history)

final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_intrinsic_env_model_"+exploration_tech+"_video.mp4")